# Basic tutorial: image data
#### Author: Matteo Caorsi

This short tutorial provides you with the basic functioning of *giotto-deep* API.

The main steps of the tutorial are the following:
 1. creation of a dataset
 2. creation of a model
 3. define metrics and losses
 4. run benchmarks
 5. visualise results interactively

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np

import torch
from torch import nn

from gdeep.models import FFNet

from gdeep.visualisation import  persistence_diagrams_of_activations

from torch.utils.tensorboard import SummaryWriter
from gdeep.data import TorchDataLoader


from gtda.diagrams import BettiCurve

from gtda.plotting import plot_betti_surfaces

import optuna

# Initialize the tensorboard writer

In order to analyse the reuslts of your models, you need to start tensorboard.
On the terminal, move inside the `/example` folder. There run the following command:

```
tensorboard --logdir=runs
```

Then go [here](http://localhost:6006/) after the training to see all the visualisation results.

In [2]:
writer = SummaryWriter()

# Create your dataset

In [8]:
from torch.utils.data.sampler import SubsetRandomSampler

dl = TorchDataLoader(name="CIFAR10")
train_indices = []
for i in range(10000):
    train_indices.append(i)

dl_tr, dl_tr = dl.build_dataloader(batch_size=1024, sampler=SubsetRandomSampler(train_indices))

test_indices = []
for i in range(3000):
    test_indices.append(i)

dl_ts, dl_ts = dl.build_dataloader(batch_size=1024, sampler=SubsetRandomSampler(test_indices))

print(len(dl_tr))
print(len(dl_ts))

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
10
3


## Define and train your model

In [9]:
import torchvision.models as models
from gdeep.pipeline import Pipeline

model = nn.Sequential(models.resnet18(pretrained=True), nn.Linear(1000,10))

In [12]:
from torch.optim import SGD, Adam, RMSprop

# print(model)
loss_fn = nn.CrossEntropyLoss()

pipe = Pipeline(model, (dl_tr, dl_ts), loss_fn, writer, False)

# train the model
pipe.train(SGD, 10, lr=0.01)
# pipe.train([SGD, Adam], 2, lr=[0.001, 0.01])



{'lr': 0.01}
<class 'torch.optim.sgd.SGD'>
TOTAL EPOCHS  10
Epoch 1
-------------------------------
Time taken for this epoch: 4s
Test Error: 
 Accuracy: 30.0%,                 Avg loss: 0.000001 

Epoch 2
-------------------------------
Time taken for this epoch: 4s
Test Error: 
 Accuracy: 30.0%,                 Avg loss: 0.000001 

Epoch 3
-------------------------------
Time taken for this epoch: 4s
Test Error: 
 Accuracy: 30.0%,                 Avg loss: 0.000001 

Epoch 4
-------------------------------
Time taken for this epoch: 4s
Test Error: 
 Accuracy: 30.0%,                 Avg loss: 0.000001 

Epoch 5
-------------------------------
Time taken for this epoch: 4s
Test Error: 
 Accuracy: 30.0%,                 Avg loss: 0.000001 

Epoch 6
-------------------------------
Time taken for this epoch: 4s
Test Error: 
 Accuracy: 30.0%,                 Avg loss: 0.000001 

Epoch 7
-------------------------------
Time taken for this epoch: 4s
Test Error: 
 Accuracy: 30.0%,            

KeyboardInterrupt: 

In [ ]:
from gdeep.pipeline import Gridsearch

loss_fn = nn.CrossEntropyLoss()

gs = Gridsearch(learning_rate = [0.001, 0.01], [Adam, SGD], model, (dl_tr, dl_ts), loss_fn, writer, False)

# train the model
gs.search(n_trials = 100)

# Simply use interpretability tools

In [ ]:
from gdeep.analysis.interpretability import Interpreter

inter = Interpreter(model, method="IntegratedGradients")
inter.interpret_image(next(iter(dl_tr))[0][0].reshape(1,3,32,32), 6);


# Extract inner data from your models

In [ ]:
from gdeep.models import ModelExtractor

me = ModelExtractor(model, loss_fn)

lista = me.get_layers_param()

for k, item in lista.items():
    print(k,item.shape)


In [ ]:
x = next(iter(dl_tr))[0][0]
if x.dtype is not torch.int64:
    res = me.get_decision_boundary(x, n_epochs=1)
    res.shape

In [ ]:
x = next(iter(dl_tr))[0]
list_activations = me.get_activations(x)
len(list_activations)


In [ ]:
x, target = next(iter(dl_tr))
if x.dtype is torch.float:
    for gradient in me.get_gradients(x, target=target)[1]:
        print(gradient.shape)

# Visualise activations and other topological aspects of your model

In [ ]:
from gdeep.visualisation import Visualiser

vs = Visualiser(pipe)

vs.plot_data_model()
vs.plot_activations(x)
vs.plot_persistence_diagrams(x)


In [ ]:
vs.plot_interpreter_image(inter);